# Plotting data from GEA
## Intro
This is an example Jupyter Notebook to show how our hacky EPICS Data Capture and Plotting python scripts can be turned into a Powerful AF (tm) web tool to analyze data.

<div class="alert alert-block alert-danger">
<b>Disclaimer!:</b> This is not meant to completely replace GEA but to be a follow-up more in-depth analysis tool.
</div>

## The awesome Real Time EPICS data analysis module
The developer's purpose with this module was to give the users some flexibility into how to plot and analyze data gathered from our EPICS systems.

The first part of the module is the utility that allows the user to get data from GEA or directly from EPICS IOCs. GEA being our engineering archive is an extensive database compiled through times immemorial. However, being able to harvest data from IOCs for a particular test or investigation without having to configure said records in GEA (!!!) is also nice.

The main script in this section, found in `./util/` is:
 - `inPosDataCap.py`

The library being used by this script, found in `./lib/` is:
 - `dataFromGea.py`
 
The second part of the module is the python library that provides the users with tools to manipulate and plot data to their hearts desire. In this library the users will find a myriad of methods/functions/utilities to filter/compare/enhance data. However, the killer functionality is the set of classes that allow for plots to be formatted and tiled without having to deal with the fun and exciting matplotlib crap (jk, matplotlib is the goat).
All goodies are found in the python file in `./lib/`:
 - `plotEpicsData.py`

<div class="alert alert-block alert-warning">
<b>Note:</b> As a bonus, the module allowed the dev to be a lazy pos that couldn't be bother to develop an actual UI, which would have been way better.
</div>

## How to gather data
As it was mentioned above, data can be harvested after the fact from GEA or capturing a live data stream by tapping directly into IOCs. Data is gathered using the script `inPosDataCap.py`. All data is saved by default in the `./data/` directory.

The utility `inPosDataCap.py` has two modes:
 - **GEA**: This mode is enabled by the argument `gea`. In its most basic form it **reads an input text file** passed as an argument, which contains a list of EPICS channels. The data is gathered for the **time window specified by the time string input** arguments with format `yymmddTHHMM`. The user also need to specify the **site from which data is being gathered** (input argument `gs` for Gemini South and `gn` for Gemini North). The data is then returned as an __[hdf5](https://www.neonscience.org/resources/learning-hub/tutorials/about-hdf5)__ file with the default prefix `recDataGea-{start_date}`. An example command would go as follow:
    ```
    $ ./util/inPosDataCap.py gea gs FastTrack-chans.txt 230412T2200 230412T2300
    
 
    ```    
    This would gather the data from **GEA South** for the channels in `FastTrack-chans.txt` and write it to the file `recDataGea-230412T2200.h5`. The file is saved to `./data`, since no custom dir was specified in the arguments.

 - **Channel Access**: This mode is enabled by the input argument `ca`. Data is collected live for the list of EPICS channels especified in the **input text file**, for the duration specified by the user (eg. -`hr 3` would set the script to capture data for 3 hours). An example command is shown below
    ```
    $ ./util/inPosDataCap.py ca BtO-chans.txt -min 45
    
    
    ```
    This would set CA monitors to gather data from the channels specified in `BtO-chans.txt` for 45 min after the execution of the command. Once the capture timer is done, the file `recMonCA-{date-of-execution}.h5` would be generated and stored in `./data`, as no user dir was specified.

In [ ]:
%matplotlib ipympl
from chanmonitor.lib.plotEpicsData import DataAx, DataAxePlotter
import chanmonitor.lib.plotEpicsData as ped

In [ ]:
hdf5File = '../data/FT-OpsNight-230417T2200.h5'
stime = ''
etime = ''

In [ ]:
# Read h5 file
recData = ped.extract_hdf5([hdf5File],
                           stime,
                           etime)

In [ ]:
mc_azPos = DataAx(recData['mc:azCurrentPos'],
                  'xkcd:bright blue',
                  label='mc:azCurrentPos',git add
                  ylabel='Position [deg]',
                  marker='o',
                  marksize=5,
                  linewidth=1.25)

In [ ]:
plts = DataAxePlotter(ncols=1)

plts.Axe['c1']['mc_azPos'] = mc_azPos
#plts.Axe['c2']['mc_azDmd'] = mc_azDmd
#plts.Axe['c2']['mc_azPos'] = DataAx.update_axe(mc_azPos,
#                                                   shaxname='mc_azDmd')
#plts.Axe['c2']['mc_azPos_tcs'] = DataAx.update_axe(mc_azPos_tcs,
#                                                   shaxname='mc_azDmd')
#plts.Axe['c2']['mc_azPos_fllw'] = DataAx.update_axe(mc_azPos_fllw,
#                                                   shaxname='mc_azDmd')
# plts.Axe['c1']['mc_azPmacDmd'] = mc_azPmacDmd
#plts.Axe['c1']['mc_azErr'] = mc_azErr
#plts.Axe['c1']['mc_azPmacErr'] = mc_azPmacErr
#plts.Axe['c1']['tcs_lost_dmd'] = tcs_lost_dmd
#plts.Axe['c1']['tcs_lost_diff'] = tcs_lost_diff

# plts.Axe['c2']['mc_elDmd'] = mc_elDmd
# plts.Axe['c2']['mc_elPos'] = DataAx.update_axe(mc_elPos,
                                                   # shaxname='mc_elDmd')
# plts.Axe['c2']['mc_elPmacDmd'] = mc_elPmacDmd
# plts.Axe['c2']['mc_elErr'] = mc_elErr
# plts.Axe['c2']['mc_elPmacErr'] = mc_elPmacErr

# plts.Axe['c3']['mc_trk_azerr_fft'] = mc_trk_azerr_fft
# plts.Axe['c3']['mc_trk_elerr_fft'] = mc_trk_elerr_fft
plts.positionPlot()
plts.plotConfig('FR-42809 Analysis')